# Workshop #2 :
## Merge

------------------------------------------------------------

Integrate Grammy, Spotify, and API data into a unified dataset. 

In [ ]:

import pandas as pd
import numpy as np
import logging

Reading clean CSVs as Dataframes for manipulation.

In [23]:
grammy = pd.read_csv("/home/bb-8/Dags/ws_002/data/intermediate/grammys.csv")
spotify = pd.read_csv("/home/bb-8/Dags/ws_002/data/intermediate/spotify.csv")
api = pd.read_csv("/home/bb-8/Dags/ws_002/data/intermediate/api.csv")

## Reviewing `features` in each dataset

In [24]:
api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   nominee           90 non-null     object 
 1   acousticness      90 non-null     float64
 2   danceability      90 non-null     float64
 3   energy            90 non-null     float64
 4   instrumentalness  90 non-null     float64
 5   liveness          90 non-null     float64
 6   loudness          90 non-null     float64
 7   speechiness       90 non-null     float64
 8   tempo             90 non-null     float64
 9   valence           90 non-null     float64
dtypes: float64(9), object(1)
memory usage: 7.2+ KB


In [25]:
grammy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4430 entries, 0 to 4429
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   year                 4430 non-null   int64 
 1   title                4430 non-null   object
 2   nominee              4430 non-null   object
 3   artist               4430 non-null   object
 4   winner               4430 non-null   bool  
 5   normalized_category  4430 non-null   object
dtypes: bool(1), int64(1), object(4)
memory usage: 177.5+ KB


In [26]:
spotify.describe(include="all")

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,minutes,mode_nominal,key_nominal
count,81158,81158,81158,81158,81158.000000,8.115800e+04,81158,81158.000000,81158.000000,81158.000000,...,81158.000000,81158.000000,81158.000000,81158.000000,81158.000000,81158.000000,81158,8.115800e+04,81158,81158
unique,81158,28157,41877,66331,NaN,NaN,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,55,NaN,2,12
top,5SuOikwiRyPMVoIQDJUgSV,George Jones,The Complete Hank Williams,Rockin' Around The Christmas Tree,NaN,NaN,No,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,punk-rock,NaN,major,G
freq,1,260,110,48,NaN,NaN,74261,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3916,NaN,51915,9544
mean,NaN,NaN,NaN,NaN,33.041179,2.271004e+05,NaN,0.565145,0.639840,5.279677,...,0.324653,0.177126,0.218445,0.476717,122.349142,3.900071,NaN,2.271004e+05,NaN,NaN
std,NaN,NaN,NaN,NaN,20.730254,1.125687e+05,NaN,0.174687,0.258049,3.557962,...,0.340479,0.325627,0.195963,0.262441,30.116940,0.451439,NaN,1.125687e+05,NaN,NaN
min,NaN,NaN,NaN,NaN,0.000000,1.745300e+04,NaN,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,1.745300e+04,NaN,NaN
25%,NaN,NaN,NaN,NaN,19.000000,1.715005e+05,NaN,0.454000,0.462000,2.000000,...,0.014600,0.000000,0.098100,0.258000,99.907250,4.000000,NaN,1.715005e+05,NaN,NaN
50%,NaN,NaN,NaN,NaN,32.000000,2.116380e+05,NaN,0.579000,0.686000,5.000000,...,0.176000,0.000074,0.133000,0.468000,122.129500,4.000000,NaN,2.116380e+05,NaN,NaN
75%,NaN,NaN,NaN,NaN,49.000000,2.616000e+05,NaN,0.693000,0.860000,8.000000,...,0.625000,0.118000,0.282000,0.690000,140.142750,4.000000,NaN,2.616000e+05,NaN,NaN


Defining a function to perform the merge.

In [27]:
def merge_and_add_nominees(df_api, df_spotify, df_grammy=None):
    """
    Merges song data from API (nominees) with Spotify tracks, preserving artist information.
    Adds unmatched songs and optionally merges Grammy winner status.
    """
    # Merge on song names (nominee = track_name)
    merged_df = pd.merge(
        df_spotify,
        df_api.rename(columns={'nominee': 'track_name'}),
        on='track_name',
        how='left',
        suffixes=('_spotify', '_api')
    )

    # List of audio features to combine
    audio_features = ['acousticness', 'danceability', 'energy',
                    'instrumentalness', 'liveness', 'loudness',
                    'speechiness', 'tempo', 'valence']

    # Combine audio features (prioritize API data)
    for feature in audio_features:
        merged_df[feature] = merged_df[f'{feature}_api'].combine_first(merged_df[f'{feature}_spotify'])

    # Clean up columns
    merged_df.drop(columns=[f'{feature}_api' for feature in audio_features], inplace=True)
    merged_df.drop(columns=[f'{feature}_spotify' for feature in audio_features], inplace=True)

    # Identify unmatched songs (nominees not in Spotify)
    unmatched_songs = df_api[~df_api['nominee'].isin(df_spotify['track_name'])].copy()
    unmatched_songs = unmatched_songs.rename(columns={'nominee': 'track_name'})

    # Preserve original Spotify columns structure
    spotify_columns = df_spotify.columns.tolist()
    for col in spotify_columns:
        if col not in unmatched_songs:
            unmatched_songs[col] = np.nan  # Fill missing Spotify data

    # Combine datasets
    final_df = pd.concat([merged_df, unmatched_songs[spotify_columns]], ignore_index=True)

    # Add Grammy winner status if available
    if df_grammy is not None:
        final_df = pd.merge(
            final_df,
            df_grammy.rename(columns={'nominee': 'track_name'}),
            on='track_name',
            how='left'
        )

    return final_df

Using the function wwith our Dataframes and printing the resulting Datafrma's shape.

In [28]:
result_df = merge_and_add_nominees(df_api=api, df_spotify=spotify, df_grammy=grammy)
print(result_df.shape)

(81797, 28)


## Reviewing nulls

In [29]:
result_df.isnull().sum() 

track_id                 126
artists                  126
album_name               126
track_name                 0
popularity               126
duration_ms              126
explicit                 126
key                      126
mode                     126
time_signature           126
track_genre              126
minutes                  126
mode_nominal             126
key_nominal              126
acousticness               0
danceability               0
energy                     0
instrumentalness           0
liveness                   0
loudness                   0
speechiness                0
tempo                      0
valence                    0
year                   79631
title                  79631
artist                 79631
winner                 79631
normalized_category    79631
dtype: int64

### Key Observations
1. **Complete Columns (0 Nulls):**
   - `track_name`, audio features (`acousticness`, `danceability`, etc.)
   - **Why:** These exist in both API and Spotify data, with API values filling Spotify gaps

2. **Spotify-Specific Nulls (126 Nulls):**
   - `artists`, `album_name`, `popularity`, `duration_ms`, etc.
   - **Cause:** These represent 126 API nominees not found in Spotify's catalog
   - **Solution:** Manual artist/title disambiguation or accept missing Spotify metadata

3. **Massive Nulls (79,631):**
   - `year`, `title`, `artist`, `winner`
   - **Root Cause:** These columns likely come from `df_grammy` and indicate:
     - Grammy data only exists for `84,374 - 79,631 = 4,743` tracks
     - Mismatched merge keys (track_name alone isn't sufficient for Grammy matching)


## Action Plan

### Set default for missing winners (False)

In [30]:
result_df['winner'] = result_df['winner'].fillna(False)

### Fix Grammy Merging

#### Fix Grammy Merging:

Merge using both track and artist for higher accuracy

In [31]:
final_df = pd.merge(
    result_df,
    grammy.rename(columns={
        'nominee': 'track_name',
        'artist': 'artists'  # Match Spotify's column name
    }),
    on=['track_name', 'artists'],
    how='left'
)

### Remove columns with >90% nulls

In [32]:
# Remove columns with >90% nulls
final_df = final_df.loc[:, final_df.isnull().mean() < 0.9]

### Prioritize Data Sources:

In [33]:
# Create data source indicator
final_df['data_source'] = np.where(
    final_df['artists'].isnull(),
    'API Only',
    'Spotify + API'
)

In [34]:
final_df.isnull().sum() 

track_id            126
artists             126
album_name          126
track_name            0
popularity          126
duration_ms         126
explicit            126
key                 126
mode                126
time_signature      126
track_genre         126
minutes             126
mode_nominal        126
key_nominal         126
acousticness          0
danceability          0
energy                0
instrumentalness      0
liveness              0
loudness              0
speechiness           0
tempo                 0
valence               0
winner_x              0
data_source           0
dtype: int64

## Reviewing nulls again

**Spotify-Specific Missing Columns (126 nulls each)**

These represent API nominees not found in Spotify. Preserve these records while handling missingness:

### 1. Column Categorization & Treatment Strategy
### Core Identifiers (Preserve Structure)

In [35]:
final_df['artists'] = final_df['artists'].fillna('Unknown Artist')
final_df['album_name'] = final_df['album_name'].fillna('Unspecified Album')

### Spotify Metadata (Impute with Meaningful Values)

In [36]:
spotify_metadata = {
    'popularity': ('Missing', -1),
    'duration_ms': ('Median', final_df['duration_ms'].median()),
    'explicit': ('Unknown', 'unknown'),
    'key': ('Missing Category', 'missing'),
    'mode': ('Missing Category', 'missing'),
    'time_signature': ('Missing Code', -1),
    'track_genre': ('Unknown Genre', 'unknown_genre')
}

for col, (strategy, value) in spotify_metadata.items():
    if strategy == 'Median':
        final_df[col] = final_df[col].fillna(value)
    else:
        final_df[col] = final_df[col].fillna(value).astype(str if isinstance(value, str) else 'category')


### C. Derived Features (Recreate from Imputed Data)

In [37]:
# Recalculate temporal features
final_df['minutes'] = (final_df['duration_ms'] / 60000).round(2)

# Re-encode categorical representations
key_mapping = {
    **{i: v for i, v in enumerate(['C', 'C♯/D♭', 'D', 'D♯/E♭', 'E', 'F',
                                  'F♯/G♭', 'G', 'G♯/A♭', 'A', 'A♯/B♭', 'B'])},
    'missing': 'Unknown'
}

mode_mapping = {
    **{0: 'Minor', 1: 'Major'},
    'missing': 'Unknown'
}

final_df['key'] = final_df['key'].map(key_mapping)
final_df['mode'] = final_df['mode'].map(mode_mapping)

### 2. Data Quality Preservation

In [38]:
# Add data source metadata
final_df['data_quality'] = np.where(
    final_df['data_source'] == 'API Only',
    'Partial Metadata (API Only)',
    'Full Metadata (Spotify + API)'
)

# Create validation flags
final_df['requires_spotify_verification'] = final_df['data_source'] == 'API Only'

### 3. Analytical Safeguards

In [39]:
# Protect core audio features
audio_features = ['acousticness', 'danceability', 'energy',
                'instrumentalness', 'liveness', 'loudness',
                'speechiness', 'tempo', 'valence']
# Column organization
final_cols = [
    # Core identifiers
    'track_name', 'artists', 'album_name', 'data_source', 'data_quality',
    
    # Spotify metadata
    'track_id', 'popularity', 'duration_ms', 'minutes', 'explicit',
    
    # Musical features
    'key_nominal', 'mode_nominal', 'time_signature',
    
    # Audio characteristics
    *audio_features,
    
    # Validation
    'requires_spotify_verification', 'winner_x'
]

final_df = final_df[final_cols]

This approach maintains the 126 API-originated records while enabling:

- Mixed-source analysis

- Clear data quality auditing

- Transparent reporting of data limitations

In [40]:
final_df = final_df.drop(['key_nominal', 'mode_nominal'], axis=1)

In [41]:
final_df.isnull().sum() 

track_name                         0
artists                            0
album_name                         0
data_source                        0
data_quality                       0
track_id                         126
popularity                         0
duration_ms                        0
minutes                            0
explicit                           0
time_signature                     0
acousticness                       0
danceability                       0
energy                             0
instrumentalness                   0
liveness                           0
loudness                           0
speechiness                        0
tempo                              0
valence                            0
requires_spotify_verification      0
winner_x                           0
dtype: int64

In [42]:
final_df.to_csv('../data/processed/final_data.csv', index=False)

logging.info("DataFrame has been saved to '../data/intermediate/api.csv'")